In [5]:
import os
import sct_utils as sct
import numpy as np
from msct_image import Image
from sct_deepseg_lesion import crop_image_around_centerline 
from sct_deepseg_lesion import apply_intensity_normalization

In [6]:
# Specifying input data
PATH_nii='/Users/pruthvi_local/Desktop/Lesion_Segmentation/Only_NIIs/'
PATH_mask = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/Masks/'
subject_masks=os.listdir(PATH_mask)
Mask_list=[x.split('.')[0] for x in subject_masks]
# print(subject_nii)
# print(subject_list)

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]

In [7]:
# # 2.1 --Second option of getting centerline -- 2.1

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_nii + subject + '.nii'
    out_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/01deepseg_sc'
    sct.run(['sct_deepseg_sc', '-i', img_path, '-c', 't2', '-ofolder', out_path], verbose=0)


PATH_deep_seg = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/01deepseg_sc/'
out_deep_seg_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/02sct_process_seg/' 

# Second option of getting centerline -- 2.2
# Specifying input data
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_deep_seg + subject + '_seg.nii'
    print(img_path) 
    try: 
        sct.run(['sct_process_segmentation', '-i', img_path, '-p', 'centerline', '-ofolder', out_deep_seg_path], verbose=0)
    except:
        continue    

In [8]:
# 3.0 Reorientation of original image and ROI

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_nii + subject + '.nii'    
    print img_path   
    try: 
        sct.run(['sct_image', '-i', img_path, ' -setorient RPI'], verbose=0)
    except:
        continue   

# Specifying roi data
PATH_process_seg = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/02sct_process_seg/'
  
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_process_seg + subject + '_seg_centerline.nii.gz'
    print img_path   
    try: 
        sct.run(['sct_image', '-i', img_path, ' -setorient RPI'], verbose=0)
    except:
        continue 


# Here specify LESION MASK DATA
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_mask + subject + '_mask.nii'
    print img_path   
    try: 
        sct.run(['sct_image', '-i', img_path, ' -setorient RPI'], verbose=0)
    except:
        continue 


In [26]:
# 4.0 Resampling of image

# Resample Input image
out_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/'
 
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_nii + subject + '.nii'
    img_out_path = out_path + subject + '_resample_INPUT.nii.gz'   
    print(img_out_path)
    try: 
        sct.run(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
    except:
        continue 
      
     
# Resample centerline mask   

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_process_seg + subject + '_seg_centerline.nii.gz'
    img_out_path = out_path + subject + '_resample_centerline.nii.gz' 
    print(img_out_path)
    try: 
#         sct.run(['sct_resample', '-i', img_path, '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
        sct.run(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)

    except:
        continue 


        
# Resample LESION MASK DATA

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_mask + subject + '_mask.nii'
    img_out_path = out_path + subject + '_resample_mask.nii.gz'   
    print(img_out_path)
    try: 
#         sct.run(['sct_resample', '-i', img_path, '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
        sct.run(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
    except:
        continue 

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aa1_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ap4_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aq0_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ar1_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ar2_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/au8_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/av9_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aw5_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ax9_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/az7_resample_INPUT.nii.gz


/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ky3_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/la9_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/lj7_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/lk2_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/lo8_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/lp0_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/md4_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/md8_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mf5_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mo9_resample_INPUT.nii.gz


/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/cj9_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ck9_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/cu8_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/cv6_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/cw2_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/cx8_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/cy7_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/dc6_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/dd8_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_pre

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/re2_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/re3_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ro5_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/rr5_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ru3_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/rw9_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/se1_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/sg6_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/sp2_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_pre

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/dt6_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/dw4_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/dz2_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ej4_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ep7_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/eq1_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/er3_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/er9_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/et5_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/fd8_resample_mask.nii.gz
/Users/pru

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vf6_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vg8_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vi9_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/we2_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/wi7_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/wt2_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/wu9_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/xo4_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/xs0_resample_mask.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/xt7_resample_mask.nii.gz
/Users/pru

FileNotFoundError: No such file: '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aw5_resample_centerline.nii.gz'

In [27]:
# 5.0 Cropping image
out_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/04crop/'
PATH_resample = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/'
PATH_process_seg = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/02sct_process_seg/'
PATH_nii='/Users/pruthvi_local/Desktop/Lesion_Segmentation/Only_NIIs/'
PATH_mask = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/Masks/'


for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    
    NII_path = PATH_resample + subject + '_resample_INPUT.nii.gz'
    MASK_path = PATH_resample + subject + '_resample_MASK.nii.gz'
    ctr_path = PATH_resample + subject + '_resample_centerline.nii.gz'  

    img_NII_path = out_path + subject + '_crop_INPUT.nii'
    img_MASK_path = out_path + subject + '_crop_MASK.nii'     
    
    try: 
        crop_image_around_centerline(NII_path, ctr_path, img_NII_path, 48)
        crop_image_around_centerline(MASK_path, ctr_path, img_MASK_path, 48)
        print(NII_path)
        print(MASK_path)
        print(ctr_path)
    except:
        continue     

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aa1_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aa1_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aa1_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ap4_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ap4_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ap4_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aq0_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aq0_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aq0_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ar1_resample_I

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/df6_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/df6_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/df6_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/do0_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/do0_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/do0_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/do9_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/do9_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/do9_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/dp0_resample_I

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hc4_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hc4_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hc4_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hc7_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hc7_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hc7_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hh8_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hh8_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hh8_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/hj8_resample_I

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mo9_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mo9_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mo9_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mq7_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mq7_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/mq7_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ni2_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ni2_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ni2_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ny6_resample_I

/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ut2_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ut2_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/ut2_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vf6_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vf6_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vf6_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vg8_resample_INPUT.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vg8_resample_MASK.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vg8_resample_centerline.nii.gz
/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/vi9_resample_I

In [28]:
# Standardize image intensities
in_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/04crop/'
ofolder_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/05Norm_Int/'
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_in_path = in_path + subject + '_crop_INPUT.nii'
    img_out_path = ofolder_path + subject + '_norm.nii'
    try: 
        apply_intensity_normalization(img_in_path, img_out_path, 't2')    
    except:
        continue        